In [3]:
import warnings
import os
import sys

import numpy as np

import cv2

import keras
import tensorflow as tf

from keras.models import model_from_json
from keras.optimizers import Adam
import keras.backend as K

warnings.filterwarnings('ignore')

In [20]:
class Predictmodels:
    
    def __init__(self):
        self.ROOT_DIR = os.path.abspath('../')
        self.models_path = os.path.join(self.ROOT_DIR, 'models')
        self.weights_path = os.path.join(self.models_path, 'weights')
        
    def load_model(self):
        K.clear_session()
        
        self.unet1_model_file = os.path.join(self.models_path, 'unet1_model.json')
        self.unet1_weight_skyline_file = os.path.join(self.weights_path, 'skyline_detection_1channel.hdf5')
        self.unet1_weight_shielding_file = os.path.join(self.weights_path, 'view_shielding_rate_1channel.hdf5')
        
        self.sky_unet = self.__unet1_model(self.unet1_model_file, self.unet1_weight_skyline_file)
        self.shield_unet = self.__unet1_model(self.unet1_model_file, self.unet1_weight_shielding_file)
        
        return self.sky_unet, self.shield_unet

    def __unet1_model(self, model_file, weight_file):
        
        K.set_image_data_format('channels_last')
        K.set_floatx('float64')

        json_file = open(model_file, "r")
        loaded_model_json = json_file.read() 
        json_file.close()

        unet = model_from_json(loaded_model_json)
        unet.load_weights(weight_file)

        unet.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics = ['mse', 'mae'])
        
        return unet
    
    def unet3_model():
        pass

In [21]:
Pm = Predictmodels()

In [22]:
model1, model2 = Pm.load_model()

# Predict Train Image

In [35]:
from flask import Flask, request, jsonify
from PIL import Image, ImageDraw
import json
import base64
from io import BytesIO

app = Flask (__name__)
 
def img_prediction(file_img):
    img = Image.open(file_img).convert('L')
    
    input_arr = np.array(img.resize((256, 256)))/255
    input_arr = input_arr.reshape((1, ) + input_arr.shape)
    
    predictions = unet.predict(input_arr)
    predictions = np.array(np.round(predictions[0] * 255, 0), dtype = 'int')
    return img, predictions

def clear_img(img, threshold= 40):
    '''
    img는 array
    '''
    clear = np.where(img < threshold, 0, img)
    clear = np.where(clear > 0, 255, clear)
    
    return clear

def y_ridge(img):
    cols = img.shape[1]
    rows = img.shape[0]

    ridge = []

    for c in range(cols):
        for r in range(rows):
            row = img[r][c]
            if row > 0:
                ridge.append([c, r])
                break

    ridge.append([cols+1, rows+1])
    ridge.append([0, rows+1])            
    ridge_array = np.array(ridge)
    
    return ridge_array

def draw_skyline(img, ridge):
    
    ridge = ridge.astype(np.int32)
    
    polyline_img = cv2.polylines(img, [ridge], False, (255, 0, 0))
    
    return polyline_img

def shielding_rate(img, contour):
    x = img.shape[1]
    y = img.shape[0]
    
    contour_area = cv2.contourArea(contour)
    shield_rate = round((contour_area/(x*y)) * 100, 4)
    
    return shield_rate

@app.route('/Dpredict', methods = ['POST'])
def predict():
    if request.method == 'POST':
        
        data = request.data
        
        img = request.files['image']
        form = data['format']
        command = data['command']
        
        if command = 'skyline_detection':
            
            input_img, prediction = img_prediction(img)
        
            clear_pred = clear_img(prediction)
            ridge = y_ridge(clear_pred)
        
            resize_img = np.array(input_img.resize((256,256)), np.uint8)
            output_img = draw_skyline(resize_img, ridge)
            rate = shielding_rate(resize_img, ridge)
        
        elif command = 'view_shielding_rate':
            
            input_img, prediction = img_prediction(img)
        
            clear_pred = clear_img(prediction)
            ridge = y_ridge(clear_pred)
        
            resize_img = np.array(input_img.resize((256,256)), np.uint8)
            output_img = draw_skyline(resize_img, ridge)
            rate = shielding_rate(resize_img, ridge)
        
        if form == 'total_array':
            output = output_img.tolist()
        elif form == 'total_base64':
            rawBytes = BytesIO()
            img_buffer = Image.fromarray(output_img.astype('uint8'))
            img_buffer.save(rawBytes, 'PNG')
            rawBytes.seek(0)
            base64_img = base64.b64encode(rawBytes.read())
            output = str(base64_img)
        else :
            print('format을 입력해주세요. 현재 지원 format parameter는 total_array, total_base64입니다.')
            
#        img_result는 output image를 원래의 사이즈로 다시 키우는 것, 화질이 너무 깨져서 다른 방법 생각해야함        
#        img_result = cv2.resize(output_img, dsize = (input_img.size[0], input_img.size[1]), interpolation=cv2.INTER_CUBIC)
         
        
    return jsonify({'output_img' : output, 'shielding_rate' : rate})

In [36]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2020 18:28:09] "POST /skylinePredict HTTP/1.1" 200 -
